In [1]:
import json
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModel
from sentence_transformers import SentenceTransformer

e:\YuyangGPT\trainer\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
INPUT_FILE = "E:\\YuyangGPT\\dataset\\cleaned_data\\cleaned_discord_messages.jsonl"
OUTPUT_FILE = "E:\\YuyangGPT\\dataset\\cleaned_data\\train_tokenized_discord_messages.jsonl"
MAX_LENGTH = 2048

In [7]:
tokenizer = AutoTokenizer.from_pretrained(
    "E:\YuyangGPT\models\minilm-custom-eos",
    use_fast=True
)


In [8]:
print("EOS token:", tokenizer.eos_token)
print("EOS token id:", tokenizer.eos_token_id)
eos_id = tokenizer.eos_token_id

EOS token: <|eos|>
EOS token id: 30522


In [9]:
model = AutoModel.from_pretrained(
    "E:\YuyangGPT\models\minilm-custom-eos"
)

In [6]:
tokenizer_dir = "E:\YuyangGPT\models\minilm-custom-eos"
model.save_pretrained(tokenizer_dir)
tokenizer.save_pretrained(tokenizer_dir)

('E:\\YuyangGPT\\models\\minilm-custom-eos\\tokenizer_config.json',
 'E:\\YuyangGPT\\models\\minilm-custom-eos\\special_tokens_map.json',
 'E:\\YuyangGPT\\models\\minilm-custom-eos\\vocab.txt',
 'E:\\YuyangGPT\\models\\minilm-custom-eos\\added_tokens.json',
 'E:\\YuyangGPT\\models\\minilm-custom-eos\\tokenizer.json')

In [10]:
def tokenize_and_append_eos(text):
    tokens = tokenizer.encode(
        text,
        add_special_tokens=False,
        truncation=True,
        max_length=MAX_LENGTH - 1  # leave space for EOS
    )
    tokens.append(eos_id)
    return tokens

In [11]:
with open(INPUT_FILE, "r", encoding="utf-8") as fin, \
     open(OUTPUT_FILE, "w", encoding="utf-8") as fout:

    for line in tqdm(fin, desc="Tokenizing"):
        record = json.loads(line)
        text = record["text"].strip()

        if not text:
            continue

        input_ids = tokenize_and_append_eos(text)
        attention_mask = [1] * len(input_ids)

        out_record = {
            "input_ids": input_ids,
            "attention_mask": attention_mask
        }

        fout.write(json.dumps(out_record) + "\n")

Tokenizing: 849it [00:02, 338.19it/s] 
